#### Importando bibliotecas necessárias

In [ ]:
import sys
sys.path.append('D:\\__case_ifood\\notebooks\\utils') # Defina aqui a pasta

In [ ]:
import importlib
import utils

# Recarregar o módulo 'utils'
importlib.reload(utils)

from utils import cities, get_sales_data_some_columns, calculate_evaluation_metrics, calculate_evaluation_metrics_delivery_time, calculate_evaluation_metrics_price_range, calculate_engagement_custormers_two_orders, calculate_engagement_custormers_three_orders, calculate_engagement_custormers_three_plus_orders, calculate_engagement_custormers_three_plus_orders_delivery_time, calculate_engagement_custormers_three_plus_orders_price_range, calculate_engagement_custormers_two_orders_delivery_time, calculate_engagement_custormers_three_orders_price_range, calculate_engagement_custormers_three_orders_delivery_time


### A) – Definição de segmentações relevantes para o teste A/B

A segmentação de usuários permite refinar a análise do impacto da campanha de cupons, identificando **grupos específicos** em que a ação foi mais ou menos eficaz. A seguir, estão os segmentos definidos com suas respectivas justificativas e critérios:

---

#### Segmentos utilizados

| Segmento                           | Justificativa Estratégica                                                                 |
|------------------------------------|--------------------------------------------------------------------------------------------|
| **Cidade de entrega**              | Diferenças regionais foram nítidas nos resultados, com destaque em Cohen's d por cidade    |
| **Faixa de preço do restaurante**     | Indica o ticket médio esperado; reflete a sensibilidade a descontos                        |
| **Tempo de entrega do restaurante**| Pode impactar a percepção de valor e satisfação com a experiência de compra                |

---

### B) Critérios utilizados por segmento

- **Cidade de entrega (`merchant_city`):**  
  Selecionadas as 9 principais cidades com maior volume de pedidos na base.

- **Faixa de preço do restaurante (`price_range`):**  
  Utilização direta da variável existente. o range do preço por restaurante pode ser relevante para buscar insights não identificados.

- **Tempo de entrega do restaurante (`delivery_time`):**  
  Criação de faixas com base em quantis:  
  - 25%  → entrega até 18 min  
  - 50%  → entrega até 34 min
  - 75%  → entrega até 66 min
  - 100% → entrega acima de 66 min

---

Essas segmentações serão utilizadas para analisar as métricas de desempenho da campanha (ARPU, engajamento), com o objetivo de identificar **grupos com maior retorno** e possíveis **oportunidades de personalização futura**.

In [ ]:
# Busca os dados necessários
df_sales = get_sales_data_some_columns()

### C) Análise dos resultados

#### 1) Por cidade de entrega

##### ARPU

In [ ]:
# Métricas iniciais 
df_evaluation_metrics_by_address_city = calculate_evaluation_metrics(df_sales, "delivery_address_city", None)

In [ ]:
# Clientes com recompras

df_customers_with_2_orders = calculate_engagement_custormers_two_orders(df_sales, "delivery_address_city", None)

df_customers_with_3_orders = calculate_engagement_custormers_three_orders(df_sales, "delivery_address_city", None)

df_customers_with_3_plus_orders = calculate_engagement_custormers_three_plus_orders(df_sales, "delivery_address_city", None)

In [ ]:
# Unindo os Dataframes

df_evaluation_metrics_by_address_city = df_evaluation_metrics_by_address_city.join(
    df_customers_with_2_orders,
    on=["is_target","delivery_address_city"],
    how="left"
).with_columns([
    (pl.col("customers_with_2_orders") / pl.col("unique_customers")).round(3).alias("percent_customers_with_2_orders")
]).join(
    df_customers_with_3_orders,
    on=["is_target","delivery_address_city"],
    how="left"
).with_columns([
    (pl.col("customers_with_3_orders") / pl.col("unique_customers")).round(3).alias("percent_customers_with_3_orders")
]).join(
    df_customers_with_3_plus_orders,
    on=["is_target","delivery_address_city"],
    how="left"
).with_columns([
    (pl.col("customers_with_3_plus_orders") / pl.col("unique_customers")).round(3).alias("percent_customers_with_3_plus_orders")
]
)

In [ ]:
df_em_pandas = df_evaluation_metrics_by_address_city.group_by("delivery_address_city").agg([
    pl.col("revenue").sum().alias("total_amount")
]).with_columns([
    (pl.col("total_amount") / pl.col("total_amount").sum()).round(3).alias("percent_revenue"),
    pl.col("total_amount").rank("ordinal",descending=True).alias("rn")
]).filter(pl.col("rn") <= 20).select("delivery_address_city","total_amount","percent_revenue").to_pandas()

df_em_pandas= df_em_pandas.sort_values(by="total_amount", ascending=False)

# Plotar gráfico
plt.figure(figsize=(10, 6))
sns.barplot(data=df_em_pandas, x="percent_revenue", y="delivery_address_city", palette="viridis")

plt.title("Total de vendas (R$ %) por cidade")
plt.xlabel("Valor vendido (%)")
plt.ylabel("Cidade")
plt.grid(axis="x", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# A partir do gráfico acima, irei considerar as primeiras 9 cidades para avaliar as métricas ARPU e Engajamento (Recompra)

cities = ["Sao Paulo", "Rio De Janeiro", "Belo Horizonte", "Curitiba", 
           "Recife", "Salvador", "Brasilia", "Fortaleza","Porto Alegre"]

In [ ]:


# Lista para armazenar os resultados
results = []

# Laço para processar cada cidade
for city in cities:
    df_city = df_evaluation_metrics_by_address_city.filter(pl.col("delivery_address_city") == city)

    serie_target = df_city.filter(pl.col("is_target") == "target")["ARPU"]
    serie_control = df_city.filter(pl.col("is_target") == "control")["ARPU"]

    # Verifica se ambas as séries têm valor
    if len(serie_target) == 1 and len(serie_control) == 1:
        arpu_target_val = serie_target.item()
        arpu_control_val = serie_control.item()

        diff_abs = arpu_target_val - arpu_control_val
        diff_pct = (arpu_target_val / arpu_control_val - 1) if arpu_control_val != 0 else None

        results.append({
            "city": city,
            "ARPU_target": round(arpu_target_val, 2),
            "ARPU_control": round(arpu_control_val, 2),
            "absolute_diff": round(diff_abs, 2),
            "percent_diff": round(diff_pct * 100, 2) if diff_pct is not None else None
        })
    else:
        print(f"Aviso: Dados ausentes para '{city}' em um dos grupos. Pulando.")
# Converter lista de resultados em DataFrame Polars
df_arpu_by_city = pl.DataFrame(results)

In [ ]:
# Avaliando a diferença absoluta do ARPU por cidade
df_arpu_by_city_pd = df_arpu_by_city.to_pandas()

# Ordenar do maior para o menor
df_arpu_by_city_pd = df_arpu_by_city_pd.sort_values(by="absolute_diff", ascending=False)

# Plotar gráfico
plt.figure(figsize=(10, 6))
sns.barplot(data=df_arpu_by_city_pd, x="absolute_diff", y="city", palette="viridis")

plt.title("Diferença Absoluta do ARPU (Teste vs. Controle) por Cidade")
plt.xlabel("Diferença Absoluta (R$)")
plt.ylabel("Cidade")
plt.grid(axis="x", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# Calculando o desvio padrão por cidade
result_std_by_city = []

for city in cities:
    df_std_arpu_city = (
        df_sales
        .filter(pl.col("delivery_address_city") == city)
        .group_by(["delivery_address_city", "is_target"])  
        .agg([
            pl.col("amount").std().alias("std_arpu")
        ])
    )

    if df_std_arpu_city.height == 2:
        grupo_test = df_std_arpu_city.filter(pl.col("is_target") == "target")
        grupo_control = df_std_arpu_city.filter(pl.col("is_target") == "control")

        std_test = grupo_test["std_arpu"].item()
        std_control = grupo_control["std_arpu"].item()

        result_std_by_city.append({
            "city": city,
            "std_test": std_test,
            "std_control": std_control
        })

df_std_arpu_by_city = pl.DataFrame(result_std_by_city)

In [ ]:
# Enriquecendo o Dataframe com o desvio padrão por cidade para calcular a métrica Cohen's D
df_evaluation_metrics_by_address_city = df_evaluation_metrics_by_address_city.join(
    df_std_arpu_by_city,
    left_on=["delivery_address_city"],
    right_on=["city"],
    how="left"
)

In [ ]:
# Calculando a estatística de Cohen's D para ARPU para medir o tamanho do efeito
# Cohen's D é uma medida do tamanho do efeito que indica a diferença entre duas médias em termos de desvios-padrão.
# É calculado como a diferença entre as médias dividida pelo desvio-padrão combinado.

results_cohens_by_city = []

for city in cities:

    arpu_target_city = df_evaluation_metrics_by_address_city.filter(pl.col("is_target") == "target", pl.col("delivery_address_city") == city)["ARPU"].item()
    arpu_control_city = df_evaluation_metrics_by_address_city.filter(pl.col("is_target") == "control", pl.col("delivery_address_city") == city)["ARPU"].item()
    sd_target_city = df_evaluation_metrics_by_address_city.filter(pl.col("is_target") == "target", pl.col("delivery_address_city") == city)["std_test"].item()
    sd_control_city = df_evaluation_metrics_by_address_city.filter(pl.col("is_target") == "control", pl.col("delivery_address_city") == city)["std_control"].item()

    cohen_d = round((arpu_target_city - arpu_control_city) / (math.sqrt(((sd_target_city**2 + sd_control_city**2) / 2))),5)

    results_cohens_by_city.append({
        "city":city,
        "cohens_d":cohen_d
    })

df_cohens_by_city = pl.DataFrame(results_cohens_by_city)

In [ ]:
# Criando uma coluna categorica para explicar a métrica Cohen's D
df_cohens_by_city = df_cohens_by_city.with_columns([
    pl.when(pl.col("cohens_d") < 0.2).then(pl.lit("Muito pequeno"))
     .when(pl.col("cohens_d") < 0.5).then(pl.lit("Pequeno"))
     .when(pl.col("cohens_d") < 0.8).then(pl.lit("Médio"))
     .otherwise(pl.lit("Grande"))
     .alias("classificacao_efeito")
])

In [ ]:
# Enriquecendo o Dataframe com os dados de Cohen's D
df_evaluation_metrics_by_address_city = df_evaluation_metrics_by_address_city.join(
    df_cohens_by_city,
    left_on="delivery_address_city",
    right_on="city",
    how="left"
)

df_arpu_by_city = df_arpu_by_city.join(
    df_cohens_by_city,
    on="city",
    how="left"
)

In [ ]:
# Inicializa uma lista para armazenar os novos valores de 'unique_customers'
updated_values = []

for city in cities:
    # Filtra os dados para a cidade e 'is_target' == 'target'
    filtered_data = df_evaluation_metrics_by_address_city.filter(
        (pl.col("is_target") == "target") & (pl.col("delivery_address_city") == city)
    )
    
    # Obtém o valor de 'unique_customers' da cidade filtrada
    if filtered_data.shape[0] > 0:
        unique_customers_value = filtered_data["unique_customers"].item()  # Pega o valor de 'unique_customers'
    else:
        unique_customers_value = 0  # Se não houver dados, atribui um valor padrão

    # Adiciona o valor à lista de valores atualizados para 'unique_customers'
    updated_values.append((city, unique_customers_value))

# Agora, atualize o df_arpu_by_city com os novos valores
for city, unique_customers_value in updated_values:
    df_arpu_by_city = df_arpu_by_city.with_columns(
        pl.when(pl.col("city") == city)
        .then(pl.lit(unique_customers_value))
        .otherwise(pl.col("unique_customers"))
        .alias("unique_customers")
    )


In [ ]:
# Calculando a receita estimada por cidade de entrega
print(df_arpu_by_city.select(pl.col("city"), pl.col("absolute_diff"), pl.col("unique_customers")).with_columns([
    ((pl.col("absolute_diff") - 15) * pl.col("unique_customers")).round(0).alias("revenue_estimated")
])
)

##### Análise de impacto financeiro por Cidade

Com base na diferença de ARPU entre os grupos *teste* e *controle*, no número de clientes impactados e no custo estimado de R$15,00 por cupom, calculamos o retorno financeiro líquido por cidade.

##### Cidades com retorno positivo relevante

Estas cidades demonstraram excelente performance, com receita incremental acima do custo da campanha:

| Cidade         | Diferença ARPU (R$) | Clientes Impactados | Receita Estimada (R$) | Cohen's d |
|----------------|---------------------|----------------------|------------------------|---------|
| **São Paulo**        | 20,32                  | 122.492               | **651.657**            | 0.34 |
| **Rio de Janeiro**   | 22,57                  | 73.805                | **558.704**            | 0.45 |
| **Brasília**         | 17,58                  | 16.292                | **42.033**             | 0.58 |

---

##### Cidades com retorno levemente negativo ou abaixo de R$20k

Nesses casos, o resultado foi próximo da neutralidade, indicando potencial de ajustes na campanha (ex: cupons menores, targeting mais refinado):

| Cidade         | Diferença ARPU (R$) | Clientes Impactados | Receita Estimada (R$) | Cohen's d |
|----------------|---------------------|----------------------|------------------------|---------|
| **Fortaleza**        | 15,74                  | 16.942                | **12.537**             | 0.64 |
| **Curitiba**        | 13,96                  | 14.526                | –15.107                | 0.51 |
| **Salvador**        | 13,72                  | 12.869                | –16.472                | 0.48 |

---

##### Cidades com retorno negativo expressivo

Apesar de efeitos estatísticos positivos, nessas cidades o custo da campanha superou o ganho adicional, sugerindo reavaliação da estratégia:

| Cidade         | Diferença ARPU (R$) | Clientes Impactados | Receita Estimada (R$) | Cohen's d |
|----------------|---------------------|----------------------|------------------------|---------|
| **Belo Horizonte**  | 10,69                  | 16.282                | –70.175                | 0.10 |
| **Recife**          | 11,01                  | 15.816                | –63.106                | 0.45 |
| **Porto Alegre**    | 11,44                  | 12.650                | –45.034                | 0.30 |

---

##### Conclusão

Embora métricas estatísticas como **Cohen’s d** tenham apontado efeito positivo em diversas regiões, a **viabilidade financeira real** varia significativamente entre cidades.

Cidades como **São Paulo e Rio de Janeiro** se destacam e podem receber maior investimento em campanhas similares. Já cidades com retorno negativo exigem **segmentações mais refinadas** ou **redução no valor do cupom**.

##### Engajamento

In [ ]:
results_diff_percentage_engagement = []

for city in cities:
    percent_customers_with_2_orders_test = df_evaluation_metrics_by_address_city.filter(pl.col("is_target") == "target",
                                                                                                     pl.col("delivery_address_city") == city)["percent_customers_with_2_orders"].item()
    percent_customers_with_2_orders_control = df_evaluation_metrics_by_address_city.filter(pl.col("is_target") == "control",
                                                                                                     pl.col("delivery_address_city") == city)["percent_customers_with_2_orders"].item()
    percent_customers_with_3_orders_test = df_evaluation_metrics_by_address_city.filter(pl.col("is_target") == "target",
                                                                                                     pl.col("delivery_address_city") == city)["percent_customers_with_3_orders"].item()
    percent_customers_with_3_orders_control = df_evaluation_metrics_by_address_city.filter(pl.col("is_target") == "control",
                                                                                                     pl.col("delivery_address_city") == city)["percent_customers_with_3_orders"].item()
    percent_customers_with_3_plus_orders_test = df_evaluation_metrics_by_address_city.filter(pl.col("is_target") == "target",
                                                                                                     pl.col("delivery_address_city") == city)["percent_customers_with_3_plus_orders"].item()
    percent_customers_with_3_plus_orders_control = df_evaluation_metrics_by_address_city.filter(pl.col("is_target") == "control",
                                                                                                     pl.col("delivery_address_city") == city)["percent_customers_with_3_plus_orders"].item()
    
    diff_percent_2_orders = (percent_customers_with_2_orders_test / percent_customers_with_2_orders_control) - 1
    diff_percent_3_orders = (percent_customers_with_3_orders_test / percent_customers_with_3_orders_control) - 1
    diff_percent_3_plus_orders = (percent_customers_with_3_plus_orders_test / percent_customers_with_3_plus_orders_control) - 1

    results_diff_percentage_engagement.append({
        "city": city,
        "diff_percent_2_orders": round(diff_percent_2_orders,3),
        "diff_percent_3_orders": round(diff_percent_3_orders,3),
        "diff_percent_3_plus_orders": round(diff_percent_3_plus_orders,3)
    })

df_diff_percentage_engagement_by_city = pl.DataFrame(results_diff_percentage_engagement)

In [ ]:
df_diff_percentage_engagement_by_city

### Análise de engajamento por Cidade

Além do aumento no ARPU, avaliamos a **diferença percentual no engajamento** entre os grupos *teste* e *controle*, considerando o percentual de clientes que realizaram:

- **2 pedidos**
- **3 pedidos**
- **Mais de 3 pedidos**

#### Diferença de engajamento (grupo teste - controle)

| Cidade           | 2 pedidos (%) | 3 pedidos (%) | +3 pedidos (%) |
|------------------|---------------|----------------|----------------|
| **Belo Horizonte** | **27,2**       | 18,3            | **22,4**        |
| **Recife**         | 20,4           | **25,0**        | 13,5            |
| **Porto Alegre**   | 13,1           | **28,6**        | 13,2            |
| **Curitiba**       | 22,5           | **24,1**        | 20,3            |
| **Salvador**       | 22,2           | **23,9**        | 19,2            |
| **Rio de Janeiro** | **22,1**       | 22,0            | 17,0            |
| **São Paulo**      | 21,0           | 19,8            | 20,1            |
| **Brasília**       | 18,2           | 14,1            | **23,2**        |
| **Fortaleza**      | 18,0           | 12,6            | 16,7            |

---

### Insights

- **Porto Alegre**, **Curitiba**, **Recife**, **Salvador** e **Belo Horizonte** mostraram **forte engajamento em 3+ pedidos**, mesmo que o retorno financeiro tenha sido menor — indicando **potencial de retenção** que pode compensar o custo no médio/longo prazo.
- **Fortaleza** teve um dos menores impactos em engajamento, apesar do alto Cohen’s d e retorno financeiro modesto — sugerindo que o efeito pode estar concentrado em poucos usuários ou em valor de pedidos.
- **Brasília** teve um dos maiores engajamentos no grupo de 4+ pedidos, o que reforça o impacto positivo da campanha nesse público.

---

### Conclusão

> Análises de engajamento por cidade mostram que a campanha de cupons teve **efeitos comportamentais positivos** em diversas regiões, mesmo onde o impacto financeiro foi menor. Esses dados ajudam a guiar decisões mais estratégicas: **onde insistir**, **onde ajustar** e **onde inovar**.



#### 2) Por range de preço

In [ ]:
# A partir do gráfico acima, irei considerar as primeiras 9 cidades para avaliar as métricas ARPU e Engajamento (Recompra)

cities = ["Sao Paulo", "Rio De Janeiro", "Belo Horizonte", "Curitiba", 
           "Recife", "Salvador", "Brasilia", "Fortaleza","Porto Alegre"]

In [ ]:
df_evaluation_metrics_by_price_range = calculate_evaluation_metrics_price_range(df_sales, cities)
df_customers_with_2_orders_by_price_range = calculate_engagement_custormers_two_orders_price_range(df_sales, cities)
df_customers_with_3_orders_by_price_range = calculate_engagement_custormers_three_orders_price_range(df_sales, cities)
df_customers_with_3_plus_orders_by_price_range = calculate_engagement_custormers_three_plus_orders_price_range(df_sales, cities)

In [ ]:
# Avaliando a distribuição do ARPU por Range de preço

df_evaluation_metrics_pr_mc_pandas_target = df_evaluation_metrics_by_price_range.group_by("is_target","price_range").agg([
    pl.col("revenue").sum().round(0).alias("revenue"),
    pl.col("ARPU").sum().alias("ARPU")
]).to_pandas()

grouped_df_target = df_evaluation_metrics_pr_mc_pandas_target.groupby(['price_range', 'is_target'], as_index=False)['ARPU'].sum()

# Plotar o gráfico
plt.figure(figsize=(10, 6))

# Usando seaborn para criar um gráfico de barras agrupadas
sns.barplot(x='price_range', y='ARPU', hue='is_target', data=grouped_df_target, ci=None, estimator=sum)

plt.title('Arpu by Price Range and Target')
plt.xlabel('Price Range')
plt.ylabel('ARPU')
plt.legend(title= 'Target')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Enriquecendo o Dataframe com informações que permitirão calcular as métricas de ARPU e Engajamento

df_evaluation_metrics_by_price_range = df_evaluation_metrics_by_price_range.join(
    df_customers_with_2_orders_by_price_range,
    on=["is_target", "price_range"],
    how="left"
).with_columns([
    (pl.col("customers_with_2_orders") / pl.col("unique_customers")).round(3).alias("percent_customers_with_2_orders")
]).join(
    df_customers_with_3_orders_by_price_range,
    on=["is_target", "price_range"],
    how="left"
).with_columns([
    (pl.col("customers_with_3_orders") / pl.col("unique_customers")).round(3).alias("percent_customers_with_3_orders")
]).join(
    df_customers_with_3_plus_orders_by_price_range,
    on=["is_target", "price_range"],
    how="left"
).with_columns(
    (pl.col("customers_with_3_plus_orders") / pl.col("unique_customers")).round(3).alias("percent_customers_with_3_plus_orders")
)

In [ ]:
# Pivotando o Dataframe para que tenha somente uma linha por grupo (Controle e Teste)
df_arpu_engagement_by_price_range = df_evaluation_metrics_by_price_range.pivot("is_target", index=["price_range"], values=["unique_customers","ARPU","percent_customers_with_2_orders","percent_customers_with_3_orders","percent_customers_with_3_plus_orders"])


# Calculando as métricas necessárias para análisar o ARPU e o Engajamento por Range de preço dos parceiros
df_arpu_engagement_by_price_range = df_arpu_engagement_by_price_range.with_columns([
    (pl.col("ARPU_target") - pl.col("ARPU_control")).round(2).alias("arpu_absolute_diff"),
    ((pl.col("ARPU_target") / pl.col("ARPU_control")) - 1).round(3).alias("arpu_percent_absolute_diff"),
    ((pl.col("percent_customers_with_2_orders_target") / pl.col("percent_customers_with_2_orders_control")) - 1).round(3).alias("percent_customers_with_2_orders_diff"),
    ((pl.col("percent_customers_with_3_orders_target") / pl.col("percent_customers_with_3_orders_control")) - 1).round(3).alias("percent_customers_with_3_orders_diff"),
    ((pl.col("percent_customers_with_3_plus_orders_target") / pl.col("percent_customers_with_3_plus_orders_control")) - 1).round(3).alias("percent_customers_with_3_plus_orders_diff")
])


In [ ]:
# Calculando o desvio padrão do ARPU para assim calcular a métrica de Cohen's D

df_std_arpu_by_price_range = (
    df_sales
    .filter(pl.col("merchant_city").is_in(cities))
    .group_by(["is_target","price_range","customer_id"])
    .agg([
        pl.col("amount").sum().alias("arpu")
    ]).group_by([
        "is_target","price_range"
    ]).agg([
        pl.col("arpu").std().alias("std_arpu")
    ])
    .pivot("is_target", index=["price_range"], values=["std_arpu"])
)

In [ ]:
# Calculando a métrica de Cohen's D para avaliar o tamanho do efeito com relação ao range de preços dos restaurantes
df_arpu_engagement_by_price_range = df_arpu_engagement_by_price_range.join(
    df_std_arpu_by_price_range,
    on=["price_range"],
    how="left"
).with_columns([
    ((pl.col("ARPU_target") - pl.col("ARPU_control")) / 
     ( (pl.col("target")**2 + pl.col("control")**2) / 2).pow(0.5)  # Calculando raiz quadrada
    ).round(5).alias("cohens_d")
])

In [ ]:

print(df_arpu_engagement_by_price_range.select("price_range",
                                                           "unique_customers_target",
                                                           "arpu_absolute_diff", 
                                                           "arpu_percent_absolute_diff",
                                                           "cohens_d",
                                                           "percent_customers_with_2_orders_diff",
                                                           "percent_customers_with_3_orders_diff",
                                                           "percent_customers_with_3_plus_orders_diff"
                                                           ).sort(["price_range"], descending=False)
    )

### Análise por Range de Preço (`price_range`)

Ao realizar a análise por `price_range`, observo uma tendência consistente: **quanto maior a faixa de preço do parceiro, maior o ARPU observado no grupo teste**. No entanto, os efeitos estatísticos (Cohen's d) se mantêm em níveis baixos, ainda que consistentes.

#### Resultados por `price_range`

| Faixa de Preço | Clientes Únicos | Δ ARPU (R$) | Δ ARPU (%) | Cohen’s d | Δ Engajamento (2) | (3) | (3+) |
|----------------|------------------|-------------|------------|-----------|-------------------|------|-------|
| 1              | 82.085           | R$ 2,88     | 4,1%       | 0.032     | 11,4%             | 11,4%| 9,6%  |
| 2              | 91.357           | R$ 3,95     | 5,5%       | 0.045     | 10,2%             | 10,2%| 12,4% |
| 3              | 176.085          | R$ 6,81     | 7,0%       | 0.053     | 14,7%             | 14,7%| 14,6% |
| 4              | 97.050           | R$ 5,13     | 4,8%       | 0.040     | 11,5%             | 11,5% | 9,5%  |
| 5              | 40.116           | R$ 8,81     | 5,3%       | 0.034     | 11,5%             | 11,5%| 6,2%  |

#### Conclusões:

- O segmento **price_range = 3** apresentou o melhor equilíbrio entre aumento de ARPU e engajamento.
- O segmento **price_range = 5**, apesar do alto ARPU, teve menor recorrência (clientes com 3+ pedidos).
- Isso indica que **segmentos de parceiros com faixa de preço intermediária tendem a responder melhor tanto em valor quanto em frequência.**


#### 3) Por tempo de entrega

In [ ]:
cities = ["Sao Paulo", "Rio De Janeiro", "Belo Horizonte", "Curitiba", 
           "Recife", "Salvador", "Brasilia", "Fortaleza","Porto Alegre"]

In [ ]:
# Buscando os dados necessários
df_evaluation_metrics_by_delivery_time = calculate_evaluation_metrics_delivery_time(df_sales, cities)
df_customers_with_2_orders_by_delivery_time = calculate_engagement_custormers_two_orders_delivery_time(df_sales, cities)
df_customers_with_3_orders_by_delivery_time = calculate_engagement_custormers_three_orders_delivery_time(df_sales, cities)
df_customers_with_3_plus_orders_by_delivery_time = calculate_engagement_custormers_three_plus_orders_delivery_time(df_sales, cities)

In [ ]:
# Validando a distribuição de ARPU por categoria de entrega e grupo de clientes (Controle e Teste)

df_evaluation_metrics_pr_mc_pandas_delivery_time = df_evaluation_metrics_by_delivery_time.group_by("is_target","delivery_time_category").agg([
    pl.col("revenue").sum().round(0).alias("revenue"),
    pl.col("ARPU").median().alias("ARPU")
]).to_pandas()

grouped_df_delivery_time = df_evaluation_metrics_pr_mc_pandas_delivery_time.groupby(['delivery_time_category', 'is_target'], as_index=False)['ARPU'].median()

# Plotar o gráfico
plt.figure(figsize=(10, 6))

# Usando seaborn para criar um gráfico de barras agrupadas
sns.barplot(x='delivery_time_category', y='ARPU', hue='is_target', data=grouped_df_delivery_time, ci=None, estimator=sum)

plt.title('Arpu by delivery time')
plt.xlabel('Delivery Time')
plt.ylabel('ARPU')
plt.legend(title= 'Target')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Enriquecendo o DataFrame com colunas que permitiram calcular as métricas de ARPU e Engajamento

df_evaluation_metrics_by_delivery_time = df_evaluation_metrics_by_delivery_time.join(
    df_customers_with_2_orders_by_delivery_time,
    on=["is_target", "delivery_time_category"],
    how="left"
).with_columns([
    (pl.col("customers_with_2_orders") / pl.col("unique_customers")).round(3).alias("percent_customers_with_2_orders")
]).join(
    df_customers_with_3_orders_by_delivery_time,
    on=["is_target", "delivery_time_category"],
    how="left"
).with_columns([
    (pl.col("customers_with_3_orders") / pl.col("unique_customers")).round(3).alias("percent_customers_with_3_orders")
]).join(
    df_customers_with_3_plus_orders_by_delivery_time,
    on=["is_target", "delivery_time_category"],
    how="left"
).with_columns(
    (pl.col("customers_with_3_plus_orders") / pl.col("unique_customers")).round(3).alias("percent_customers_with_3_plus_orders")
)

In [ ]:
# Pivotando o dataframe para ter uma linha por perfil (Controle e Teste)

df_arpu_engagement_by_delivery_time = df_evaluation_metrics_by_delivery_time.pivot("is_target", index=["delivery_time_category"], values=["unique_customers","ARPU","percent_customers_with_2_orders","percent_customers_with_3_orders","percent_customers_with_3_plus_orders"])


# Calculando métricas como diferença do ARPU e seu percentual e a diferença percentual entre clientes com 2, 3 ou mais de 3 pedidos

df_arpu_engagement_by_delivery_time = df_arpu_engagement_by_delivery_time.with_columns([
    (pl.col("ARPU_target") - pl.col("ARPU_control")).round(2).alias("arpu_absolute_diff"),
    ((pl.col("ARPU_target") / pl.col("ARPU_control")) - 1).round(3).alias("arpu_percent_absolute_diff"),
    ((pl.col("percent_customers_with_2_orders_target") / pl.col("percent_customers_with_2_orders_control")) - 1).round(3).alias("percent_customers_with_2_orders_diff"),
    ((pl.col("percent_customers_with_3_orders_target") / pl.col("percent_customers_with_3_orders_control")) - 1).round(3).alias("percent_customers_with_3_orders_diff"),
    ((pl.col("percent_customers_with_3_plus_orders_target") / pl.col("percent_customers_with_3_plus_orders_control")) - 1).round(3).alias("percent_customers_with_3_plus_orders_diff")
])


In [ ]:
# Calculando o Desvio Padrão do ARPU do cliente para então Calcular a medida de Cohen's D

df_std_arpu_by_delivery_time = (
    df_sales
    .filter(pl.col("merchant_city").is_in(cities))
    .with_columns([
        pl.when(pl.col("delivery_time") <= pl.col("delivery_time").quantile(0.25)).then(pl.lit("Q1"))
        .when(pl.col("delivery_time") <= pl.col("delivery_time").quantile(0.5)).then(pl.lit("Q2"))
        .when(pl.col("delivery_time") <= pl.col("delivery_time").quantile(0.75)).then(pl.lit("Q3"))
        .otherwise(pl.lit("Q4"))
        .alias("delivery_time_category")
    ])
    .group_by(["is_target","delivery_time_category", "customer_id"])
    .agg([
        pl.col("amount").sum().alias("arpu")
    ])
    .group_by([
        "is_target","delivery_time_category"
    ]).agg([
        pl.col("arpu").std().alias("std_arpu")
    ])
    .pivot("is_target", index=["delivery_time_category"], values=["std_arpu"])
)

In [ ]:
# Calculando a medida Cohen'S D para verificar a tamanho do efeito para as categorias de entrega dos restaurantes

df_arpu_engagement_by_delivery_time = df_arpu_engagement_by_delivery_time.join(
    df_std_arpu_by_delivery_time,
    on=["delivery_time_category"],
    how="left"
).with_columns([
    ((pl.col("ARPU_target") - pl.col("ARPU_control")) / 
     ( (pl.col("target")**2 + pl.col("control")**2) / 2).pow(0.5)  # Calculando raiz quadrada
    ).round(5).alias("cohens_d")
])

In [ ]:
print(df_arpu_engagement_by_delivery_time.select("delivery_time_category",
                                                           "unique_customers_target",
                                                           "arpu_absolute_diff", 
                                                           "arpu_percent_absolute_diff",
                                                           "cohens_d",
                                                           "percent_customers_with_2_orders_diff",
                                                           "percent_customers_with_3_orders_diff",
                                                           "percent_customers_with_3_plus_orders_diff"
                                                           ).sort(["delivery_time_category"], descending=False)
    )

###  Análise por tempo de Entrega:
**Quartis**:
  - **Q1**: Entrega até 18 minutos
  - **Q2**: Entrega até 34 minutos
  - **Q3**: Entrega até 66 minutos
  - **Q1**: Entrega acima de 66 minutos
  ---

| Quartil | Clientes (target) | ARPU Δ (R$) | Δ (%) | Cohen's d | % com 2 pedidos | % com 3 pedidos | % com 3+ pedidos |
|---------|--------------------|-------------|--------|-------------|------------------|------------------|------------------|
| **Q1**  | 186.468            | 7,88        | 7,4%   | 0.056       | 14,6%           | 14,9%           | 12,0%           |
| **Q2**  | 34.932             | 7,01        | 5,6%   | 0.037       | 13,5%           | 6,5%            | 11,3%           |
| **Q3**  | 141.322            | 6,18        | 6,4%   | 0.039       | 14,5%           | 12,7%           | 13,6%           |
| **Q4**  | 93.775             | 4,50        | 5,1%   | 0.040       | 8,8%            | 9,7%            | 12,6%           |

---

### Principais Insights

1. O **ARPU absoluto** e **percentual** caem conforme aumenta o tempo de entrega (do Q1 para Q4), mas todos os quartis ainda apresentam ganhos positivos, indicando que a campanha foi efetiva mesmo em cenários operacionais distintos.

2. O **Cohen’s d** é pequeno em todos os grupos, mas consistente, sugerindo efeito estatístico modesto, porém potencialmente relevante financeiramente, principalmente dado o grande volume de clientes em Q1 e Q3.

3. O **engajamento** dos clientes (medido pela porcentagem com 2, 3 e mais de 3 pedidos) também apresenta melhores resultados em Q1 e Q3. No Q4, observa-se uma queda mais acentuada, principalmente no percentual com exatamente 2 pedidos.

---

### Conclusão

- A **eficácia da campanha** é mais expressiva entre restaurantes com entregas mais rápidas (Q1 e Q3), tanto em termos de retorno (ARPU) quanto de engajamento.
- Embora Q4 ainda apresente resultados positivos, o impacto é menor, possivelmente devido a limitações logísticas ou menor qualidade da experiência.
- **Recomendação:** priorizar campanhas promocionais em restaurantes com tempos de entrega no intervalo de Q1 a Q3 para maximizar ROI e fidelização.

In [ ]:
# Após a análise, reiniciar o kernel
%reset -f